In [113]:
from IOHexperimenter import IOH_function, IOH_logger, IOHexperimenter
import numpy as np
import sys
import heapq

In [118]:
budget = 10000

np.random.seed(123)

# hyperparameters
npop = 50 # population size 
beta = np.pi/36 # 5 degrees


In [31]:
class dotdict(dict):
    def __getattr__(self, name):
        return self[name]

In [ ]:
def ES_1(problem):
    """
    This function uses:
        - Global Sigma Mutation
        - Intermediate Recombination
        - (mu + lambda) selection
    """
    n = problem.number_of_variables
    
    lr = 1/np.sqrt(n) # learning rate (tao 0)
    lamda = npop * 7 # offspring population size
    
    fopt = -sys.maxsize-1
    
    # Initial Population samples uniformly distributed over the interval (boundaries)
    P = np.random.uniform(low=problem.lowerbound[0],high=problem.upperbound[0], size=(npop, n))
    
    sigma = (problem.upperbound[0]-problem.lowerbound[0])/6 # Global Sigma initialization with feasible range
    
    # Fitness evaluation of the population
    fitness = np.apply_along_axis(problem, 1, P)
    
    if np.max(fitness) >= fopt:
        x_prime = P[np.argmax(fitness)]
        fopt = np.max(fitness)
    
    
    ## !! final_target_hit returns True if the optimum has been found.
    ## !! evaluations returns the number of function evaluations has been done on the problem. 
    while not problem.final_target_hit and problem.evaluations < budget * n: 
        OP = [] # offspring population
        
        ### Intermediate Recombination
        for i in range(lamda):
            parent1 = P[np.random.choice(npop)]
            parent2 = P[np.random.choice(npop)]
            offspring = np.mean((parent1, parent2), axis=0)
            OP.append(offspring)
        
        OP = np.array(OP) # Offspring population
        OP_prime = []
        
        ### Mutation
        for x_i in OP:
            sigma_prime = sigma * np.exp(np.random.normal(0, lr))
            x_i_prime = x_i + np.random.normal(0, sigma_prime) 
            OP_prime.append(x_i_prime)
            
        OP_prime = np.array(OP_prime)
        
        ### Evaluation of (OP_prime + P)
        total_pop = np.append(P,OP_prime, axis=0)
        fitness = np.apply_along_axis(problem, 1, total_pop)
        if np.max(fitness) >= fopt:
            x_prime = total_pop[np.argmax(fitness)]
            fopt = np.max(fitness)
            
        ### Selection
        top_fits = heapq.nlargest(npop, fitness)
        sorter = np.argsort(fitness)
        indices = sorter[np.searchsorted(fitness, top_fits, sorter=sorter)]
        P = total_pop[indices]

    return x_prime, fopt

In [114]:
def ES_2(problem):
    """
    This function uses:
        - Individual Sigma Mutation
        - Intermediate Recombination
        - (mu + lambda) selection
    """
    n = problem.number_of_variables
    
    tao_prime = 1/np.sqrt(2*n) # global learning rate
    tao_0 = 1/np.sqrt(2*np.sqrt(n)) # local learning rate
    
    lamda = npop * 7 # offspring population size
    
    fopt = -sys.maxsize-1
    
    # Initial Population samples uniformly distributed over the interval (boundaries)
    P = np.random.uniform(low=problem.lowerbound[0],high=problem.upperbound[0], size=(npop, n))
    
    sigmas = [(problem.upperbound[0]-problem.lowerbound[0])/6]*npop # Individual sigma initialization with feasible range
    
    # Fitness evaluation of the population
    fitness = np.apply_along_axis(problem, 1, P)
    
    if np.max(fitness) >= fopt:
        x_prime = P[np.argmax(fitness)]
        fopt = np.max(fitness)
    
    
    ## !! final_target_hit returns True if the optimum has been found.
    ## !! evaluations returns the number of function evaluations has been done on the problem. 
    while not problem.final_target_hit and problem.evaluations < budget * n: 
        OP = [] # offspring population
        
        ### Intermediate Recombination
        for i in range(lamda):
            parent1 = P[np.random.choice(npop)]
            parent2 = P[np.random.choice(npop)]
            offspring = np.mean((parent1, parent2), axis=0)
            OP.append(offspring)
        
        OP = np.array(OP) # Offspring population
        OP_prime = []
        sigmas_prime = []
        
        N_tao_prime = np.random.normal(0, tao_prime)
        
        ### Mutation
        for x_i, sigma_i in zip(OP, sigmas):
            sigma_i_prime = sigma_i * np.exp(N_tao_prime + np.random.normal(0, tao_0))
            sigmas_prime.append(sigma_i_prime)
            x_i_prime = x_i + np.random.normal(0, sigma_prime) 
            OP_prime.append(x_i_prime)
            
        OP_prime = np.array(OP_prime)
        sigmas = sigmas_prime
        
        ### Evaluation of (OP_prime + P)
        total_pop = np.append(P,OP_prime, axis=0)
        fitness = np.apply_along_axis(problem, 1, total_pop)
        if np.max(fitness) >= fopt:
            x_prime = total_pop[np.argmax(fitness)]
            fopt = np.max(fitness)
            
        ### Selection
        top_fits = heapq.nlargest(npop, fitness)
        sorter = np.argsort(fitness)
        indices = sorter[np.searchsorted(fitness, top_fits, sorter=sorter)]
        P = total_pop[indices]

    return x_prime, fopt

In [ ]:
def ES_3(problem):
    """
    This function uses:
        - Correlated Mutation
        - Intermediate Recombination
        - (mu, lambda) selection
    """
    n = problem.number_of_variables
    n_q = int(n*(n-1)/2)
    
    tao_prime = 1/np.sqrt(2*n) # global learning rate
    tao_0 = 1/np.sqrt(2*np.sqrt(n)) # local learning rate 
    
    lamda = npop * 7 # offspring population size
    
    fopt = -sys.maxsize-1
    
    # Initial Population samples uniformly distributed over the interval (boundaries)
    P = np.random.uniform(low=problem.lowerbound[0],high=problem.upperbound[0], size=(npop, n)) 
    
    alphas = np.random.uniform(low=-np.pi, high=np.pi, size=n_q) # rotation angles initialized
    
    OOB_correction = lambda x: x - 2*np.pi*(x/np.abs(x)) if np.abs(x) > np.pi else x # out of boundary correction function
    
    sigmas = [(problem.upperbound[0]-problem.lowerbound[0])/6]*npop # Individual sigma initialization with feasible range
    
    # Fitness evaluation of the population
    fitness = np.apply_along_axis(problem, 1, P)
    
    if np.max(fitness) >= fopt:
        x_prime = P[np.argmax(fitness)]
        fopt = np.max(fitness)
    
    
    ## !! final_target_hit returns True if the optimum has been found.
    ## !! evaluations returns the number of function evaluations has been done on the problem. 
    while not problem.final_target_hit and problem.evaluations < budget * n: 
        OP = [] # offspring population
        
        ### Intermediate Recombination
        for i in range(lamda):
            parent1 = P[np.random.choice(npop)]
            parent2 = P[np.random.choice(npop)]
            offspring = np.mean((parent1, parent2), axis=0)
            OP.append(offspring)
        
        OP = np.array(OP) # Offspring population
        
        N_tao_prime = np.random.normal(0, tao_prime)
        
        ### Mutation
        # Step size update
        sigmas_prime = [sigma * np.exp(N_tao_prime + np.random.normal(0, tao_0)) for sigma in sigmas]
        
        # Rotation angles update
        alphas_prime = np.apply_along_axis(OOB_correction, 0, alphas + np.random.normal(0, beta, size=n_q))   
        
        # Correlation of steps sizes
        for k in range(1,n-1):
            n_i = n-k
            n_ii = n
            for i in range(1,k):
                d1, d2 = sigmas_prime[n_i], sigmas_prime[n_ii]
                print(n_q, alphas.shape, alphas_prime.shape) #alphas_prime[n_q])
                sigmas_prime[n_ii] = d1*np.sin(alphas_prime[n_q]) + d2*np.cos(alphas_prime[n_q])
                sigmas_prime[n_i] = d1*np.cos(alphas_prime[n_q]) - d2*np.sin(alphas_prime[n_q])
                n_ii = n_ii-1
                n_q = n_q-1
                
        OP_prime = []
        for x_i, sigma_i in zip(OP, sigmas_prime):
            x_i_prime = x_i + sigma_i*np.random.normal(0, sigma_i)
            OP_prime.append(x_i_prime)  
        OP_prime = np.array(OP_prime)
        
        
        ### Evaluation of OP_prime
        fitness = np.apply_along_axis(problem, 1, OP_prime)
        if np.max(fitness) >= fopt:
            x_prime = OP_prime[np.argmax(fitness)]
            fopt = np.max(fitness)
        
        ### Selection
        top_fits = heapq.nlargest(npop, fitness)
        sorter = np.argsort(fitness)
        indices = sorter[np.searchsorted(fitness, top_fits, sorter=sorter)]
        P = OP_prime[indices]

    return x_prime, fopt

In [ ]:
if __name__ == '__main__':

    ## Declarations of Ids, instances, and dimensions that the problems to be tested.
    problem_id = range(1,25)
    instance_id = range(1,26)
    dimension = [2,5,20]

    ## Declariation of IOHprofiler_csv_logger.
    ## 'result' is the name of output folder.
    ## 'studentname1_studentname2' represents algorithm name and algorithm info, which will be caption of the algorithm in IOHanalyzer.
    logger = IOH_logger("./", "result", "likhi_bozik", "likhi_bozik")

    for p_id in problem_id :
        for d in dimension :
            for i_id in instance_id:
                ## Getting the problem with corresponding id,dimension, and instance.
                f = IOH_function(p_id, d, i_id, suite="BBOB")
                f.add_logger(logger)
                xopt, fopt = likhi_bozik_ES(f)
    logger.clear_logger()


In [3]:
f = IOH_function(1, 2, 1, suite="BBOB")

In [4]:
f.get_target()

79.48

In [19]:
print(f.lowerbound[0], f.upperbound)

-5.0 [5. 5.]


In [16]:
f.number_of_variables

2

In [17]:
x = np.random.rand(f.number_of_variables) * 10 - 5
x

array([ 4.39690233, -0.82439361])

In [18]:
f(x)

96.76407811513423

In [29]:
p = np.random.uniform(low=f.lowerbound[0],high=f.upperbound[0], size=(5,f.number_of_variables))

In [30]:
np.apply_along_axis(f, 1, p)

array([ 83.37451803,  91.17468502,  83.40137455, 109.2571958 ,
        79.68275551])

In [32]:
p = dotdict()

In [39]:
p = [dotdict({'individual':[0.12, -4.54], 'strategy':[0.1, 32]}), 
     dotdict({'individual':[3.12, -1.54], 'strategy':[0.3, 2]})]

In [40]:
p

[{'individual': [0.12, -4.54], 'strategy': [0.1, 32]},
 {'individual': [3.12, -1.54], 'strategy': [0.3, 2]}]

In [42]:
p[0].individual

[0.12, -4.54]

In [43]:
p[0].strategy

[0.1, 32]

In [56]:
np.random.choice(50)

27

In [82]:
P1 = np.random.uniform(low=f.lowerbound[0],high=f.upperbound[0], size=(5, 2))

P2 = np.random.uniform(low=f.lowerbound[0],high=f.upperbound[0], size=(5, 2))

In [83]:
P1, P2

(array([[-1.36895933,  3.54972816],
        [ 2.11391802, -1.07055589],
        [-2.68698519, -1.19825286],
        [ 0.49162103,  0.56719065],
        [-4.95865357,  1.38022518]]),
 array([[-4.42351985, -4.56973093],
        [ 3.75051134, -2.07412396],
        [ 2.62767659, -1.3213474 ],
        [ 3.73502266, -4.70576266],
        [ 0.52043722, -2.59752496]]))

In [87]:
np.append(P1,P2, axis=0)

array([[-1.36895933,  3.54972816],
       [ 2.11391802, -1.07055589],
       [-2.68698519, -1.19825286],
       [ 0.49162103,  0.56719065],
       [-4.95865357,  1.38022518],
       [-4.42351985, -4.56973093],
       [ 3.75051134, -2.07412396],
       [ 2.62767659, -1.3213474 ],
       [ 3.73502266, -4.70576266],
       [ 0.52043722, -2.59752496]])

In [70]:
parent1 = P[np.random.choice(5)]
parent2 = P[np.random.choice(5)]
print(parent1, parent2)

[-2.65487125  4.87995287] [3.898657   2.50378705]


In [71]:
offspring = np.mean((parent1, parent2), axis=0)
offspring

array([0.62189287, 3.69186996])

In [72]:
(-2.65487125 + 3.898657)/2

0.6218928750000001

In [88]:
P1

array([[-1.36895933,  3.54972816],
       [ 2.11391802, -1.07055589],
       [-2.68698519, -1.19825286],
       [ 0.49162103,  0.56719065],
       [-4.95865357,  1.38022518]])

In [89]:
for i in P1:
    print(i)

[-1.36895933  3.54972816]
[ 2.11391802 -1.07055589]
[-2.68698519 -1.19825286]
[0.49162103 0.56719065]
[-4.95865357  1.38022518]


In [91]:
lr = 1/np.sqrt(2)
lr

0.7071067811865475

In [98]:
np.exp(np.random.normal(0, lr))

2.038795352227467

In [97]:
np.random.normal(0, lr)

0.9174369416643127

In [99]:
import heapq

In [103]:
P1

array([[-1.36895933,  3.54972816],
       [ 2.11391802, -1.07055589],
       [-2.68698519, -1.19825286],
       [ 0.49162103,  0.56719065],
       [-4.95865357,  1.38022518]])

In [104]:
f1 = np.apply_along_axis(f, 1, P1)
f1

array([104.26151062,  82.95119832,  88.12405528,  82.50917923,
       113.0757451 ])

In [105]:
heapq.nlargest(3, f1)

[113.07574509619684, 104.26151062275416, 88.12405528432528]

In [111]:
sorter = np.argsort(f1)
sorter[np.searchsorted(f1, heapq.nlargest(3, f1), sorter=sorter)]

array([4, 0, 2])

In [110]:
a = np.array([1, 2, 4])
b = np.array([4, 2, 3, 1])
sorter = np.argsort(b)
sorter[np.searchsorted(b, a, sorter=sorter)]

array([3, 1, 0])

In [112]:
b[[3,1,0]]

array([1, 2, 4])

In [120]:
n = f.number_of_variables

In [145]:
n = 3
beta = np.pi/36

In [147]:
alphas = np.random.uniform(low=-np.pi, high=np.pi, size=int(n*(n-1)/2))
alphas

array([-1.13872413,  1.20618494,  0.34170004])

In [150]:
samples = np.random.normal(0, beta, size=int(n*(n-1)/2))
samples

array([-0.00582568, -0.01361914,  0.13818337])

In [151]:
alphas + samples

array([-1.14454981,  1.1925658 ,  0.4798834 ])

In [158]:
func = lambda x: x - 1

In [159]:
np.apply_along_axis(func, 0, alphas + samples)

array([-2.14454981,  0.1925658 , -0.5201166 ])

In [167]:
func2 = lambda x: x - 2*np.pi*(x/np.abs(x)) if np.abs(x) > np.pi else x

In [168]:
func2(3.5)

-2.7831853071795862

In [169]:
np.pi

3.141592653589793

In [170]:
np.abs(3.5) > np.pi

True

In [171]:
3.5 - 2*np.pi*3.5

-18.491148575128552

In [179]:
for i in range(1,n):
    print(i)

1
2


In [178]:
n

3